In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as pn
import seaborn as sns 

import utils
from sklearn.metrics.pairwise import cosine_similarity, paired_distances
from sklearn import metrics 
from scipy.spatial import distance


In [2]:
subj = utils.ExpInfo.getSubjIDs()[1]
task = utils.ExpInfo.taskName[1]


In [3]:
xhy = utils.LoadData.xhy(subj, task)
xhy_disp = utils.LoadData.xhy_disp(subj, task)
d = utils.LoadData.mouseMovement_array(subj, task, velocity=True)
df_beh = utils.LoadData.behaviorData(subj, task) 

In [8]:
# iTrial = 46
dist_measure = 'euclidean'
dists = []
dist_coss = []
roc_auc = []
for iTrial in range(len(xhy[1])):
    dist_cos = []
    dist = []
    for iDot in range(3):
        h1 = xhy[1][iTrial][:-1, :]
        h2 = xhy_disp[1][iTrial][iDot]
        dist_cos_ = cosine_similarity(h1, h2).diagonal()
        dist_ = paired_distances(h1, h2, metric=dist_measure)
        dist_cos.append(dist_cos_)
        dist.append(dist_)
        
    dist = np.vstack(dist).T
    dist_cos = np.vstack(dist_cos).T    
    
    dists.append(dist)
    dist_coss.append(dist_cos)
    
    # ROC
    y_true = np.zeros(dist.shape)
    target = df_beh['target dot'][iTrial]-1
    y_true[:, target] = 1
    fpr, tpr, thresholds = metrics.roc_curve(y_true.flatten(), 1-dist.flatten())
    roc_auc.append(metrics.auc(fpr, tpr))
    
    # metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot()

In [10]:
df_beh_ = df_beh.copy()
df_beh_['AUC'] = roc_auc

In [ ]:
dists

0.5796440972222222